In [5]:
# import required modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import PoissonRegressor
from sklearn.preprocessing import PolynomialFeatures
from matplotlib import pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split

import statsmodels.api as sm



In [7]:
## import data frames

fullData = pd.read_csv("data/31DayDataUpdate.csv")
myData = fullData[['Quarter', 'HB', 'CancerType','NumberOfEligibleReferrals31DayStandard','NumberOfEligibleReferralsTreatedWithin31Days']]
myData = myData.groupby(['Quarter', 'HB', 'CancerType']).sum().reset_index()

fullData = pd.read_csv("data/62DayDataUpdate.csv")
myData2 = fullData[['Quarter', 'HB', 'CancerType','NumberOfEligibleReferrals62DayStandard','NumberOfEligibleReferralsTreatedWithin62Days']]
myData2 = myData2.groupby(['Quarter', 'HB', 'CancerType']).sum().reset_index()

weekdata = pd.read_csv("data/new_cancerdata.csv")

In [8]:
## edit 31 days data

df=myData

# remove Nans
df = df[df['NumberOfEligibleReferrals31DayStandard'].notna()]
df = df[df['NumberOfEligibleReferralsTreatedWithin31Days'].notna()]

# remove 'all cancer types' from data
df = df[df['CancerType'] != 'All Cancer Types']

# add a numerical variable 1-1 to quarters and an index variable before/after pandemic
quarters=df['Quarter']
quars=df['Quarter'].unique()
dic = {}
for i in range(len(quars)):
    dic[quars[i]]=i
dates=[dic[key] for key in quarters]
pandemic = dates.copy()
pandemic = [0 if i < dic['2020Q1'] else (i-dic['2020Q1']) for i in pandemic]        

df['Date'] =dates
df['Pandemic'] =pandemic

df.to_csv('../../31DayDataModified.csv', index=False)

## do the same with 62 days data

df1=myData2
df1 = df1[df1['NumberOfEligibleReferrals62DayStandard'].notna()]
df1 = df1[df1['NumberOfEligibleReferralsTreatedWithin62Days'].notna()]

df1 = df1[df1['CancerType'] != 'All Cancer Types']
df1 = df1[df1['HB'] != 'S92000003']

quarters=df1['Quarter']
quars=df1['Quarter'].unique()
dic = {}
for i in range(len(quars)):
    dic[quars[i]]=i
dates=[dic[key] for key in quarters]
pandemic = dates.copy()
pandemic = [0 if i < dic['2020Q1'] else (i-dic['2020Q1']) for i in pandemic]        

df1['Date'] =dates
df1['DateSquared'] = [i**2 for i in dates]
df1['Pandemic'] =pandemic

df1.to_csv('../../62DayDataModified.csv', index=False)

## edit weekly data

df2=weekdata
df2 = df2[df2['Count'].notna()]

months=np.asarray(df2['Month'])
date=np.zeros(len(months))
quars=df2['Month'].unique()
dates=np.arange(len(quars))
pandemic=np.zeros(len(months))
x=np.where(quars==202002)

for i in np.arange(len(months)):
    y=np.where(quars==months[i])
    date[i]=dates[y]
    if dates[y]>x:
        pandemic[i]=dates[y]-x
        
        
df2['Date'] =date.tolist()

df2['Pandemic'] =pandemic.tolist()


df2 = df2[df2['CancerType'] != 'All Cancers']
df2 = df2[df2['Age Group'] != 'All Ages']
df2 = df2[df2['Sex'] != 'All']
